In [8]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [9]:
HEADER = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}
url_list = [
    'https://movie.douban.com/people/98909222/wish?start=0&sort=time&rating=all&filter=all&mode=list',
    'https://movie.douban.com/people/98909222/wish?start=30&sort=time&rating=all&filter=all&mode=list',
    'https://movie.douban.com/people/98909222/wish?start=60&sort=time&rating=all&filter=all&mode=list'
]

In [10]:
def get_movie_info(movie_url):
    r_movie = requests.get(movie_url, headers=HEADER)
    soup_movie = BeautifulSoup(r_movie.text, 'html.parser')
    return {
        'name': str(soup_movie.find_all('span', {'property': 'v:itemreviewed'})[0].contents[0]),
        'year': int(soup_movie.find_all('span', {'class': 'year'})[0].contents[0].replace('(', '').replace(')', '')),
        'rating': float(soup_movie.find_all('strong', {'class': 'll rating_num', 'property': 'v:average'})[0].contents[0]),
        'votes': int(soup_movie.find_all('span', {'property': 'v:votes'})[0].contents[0])
    }

In [11]:
def get_page_info(url):
    r_page = requests.get(url, headers=HEADER)
    soup_page = BeautifulSoup(r_page.text, 'html.parser')  # 'lxml'
    movies = soup_page.find_all('div', {'class': 'title'})
    for i in range(0, len(movies)):
        movies_list.append(get_movie_info(movies[i].contents[1].attrs['href']))
        print('No. ' + str(i))
    del i

In [12]:
movies_list = []
for i in range(0, len(url_list)):
    get_page_info(url_list[i])
del i

No. 0
No. 1
No. 2
No. 3
No. 4
No. 5
No. 6
No. 7
No. 8
No. 9
No. 10
No. 11
No. 12
No. 13
No. 14
No. 15
No. 16
No. 17
No. 18
No. 19
No. 20
No. 21
No. 22
No. 23
No. 24
No. 25
No. 26
No. 27
No. 28
No. 29
No. 0
No. 1
No. 2
No. 3
No. 4
No. 5
No. 6
No. 7
No. 8
No. 9
No. 10
No. 11
No. 12
No. 13
No. 14
No. 15
No. 16
No. 17
No. 18
No. 19
No. 20
No. 21
No. 22
No. 23
No. 24
No. 25
No. 26
No. 27
No. 28
No. 29
No. 0
No. 1


In [13]:
movies_df = pd.DataFrame(data=movies_list)
movies_df['rating_year'] = movies_df['rating'] * movies_df['year']
movies_df['rating_votes'] = movies_df['rating'] * movies_df['votes']
movies_df['rating_year_votes'] = movies_df['rating'] * movies_df['year'] * movies_df['votes']

In [15]:
pd.options.display.max_rows = None
movies_df.sort_values('rating_year_votes', ascending=False).reset_index()

,index,name,year,rating,votes,rating_year,rating_votes,rating_year_votes
0,38,头号玩家 Ready Player One,2018,8.7,1035576,17556.6,9009511.2,1.818119e+10
1,58,教父 The Godfather,1972,9.3,684550,18339.6,6366315.0,1.255437e+10
2,57,教父2 The Godfather: Part Ⅱ,1974,9.2,379484,18160.8,3491252.8,6.891733e+09
3,29,雪国列车 설국열차,2013,7.5,333648,15097.5,2502360.0,5.037251e+09
4,28,E.T. 外星人 E.T.: The Extra-Terrestrial,1982,8.6,264755,17045.2,2276893.0,4.512802e+09
5,55,教父3 The Godfather: Part III,1990,8.9,253307,17711.0,2254432.3,4.486320e+09
6,51,华尔街之狼 The Wolf of Wall Street,2013,7.9,275299,15902.7,2174862.1,4.377997e+09
7,23,侏罗纪世界2 Jurassic World: Fallen Kingdom,2018,6.7,315455,13520.6,2113548.5,4.265141e+09
8,25,绿里奇迹 The Green Mile,1999,8.8,206331,17591.2,1815712.8,3.629610e+09
9,48,奇幻森林 The Jungle Book,2016,7.8,219836,15724.8,1714720.8,3.456877e+09


In [ ]:
21	18	老爷车 Gran Torino
40	16	完美风暴 The Perfect Storm